# ANALYZE EXPERIMENTS

In [1]:
import smarte.constants as cn
import smarte as smt

from dask.distributed import Client
import dask.dataframe as dd
import dask
import os

In [2]:
client = Client(n_workers=4)
client

Client Scheduler: tcp://127.0.0.1:44329 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 4 Memory: 8.21 GB


# Create the dataframe of experiments

In [3]:
DF = dd.read_csv(os.path.join(cn.EXPERIMENT_DIR, "*.csv"))
DF

,biomodel_num,num_species,num_reaction,num_parameter,method,noise_mag,range_min_frac,range_max_frac,range_initial_frac,columns_deleted,max_fev,ts_instance,median_err,max_err,min_err,avg_time,cnt,tot_time,status
npartitions=7,,,,,,,,,,,,,,,,,,,
,int64,float64,float64,float64,object,float64,float64,int64,float64,int64,int64,int64,float64,float64,float64,float64,float64,float64,object
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [4]:
path = os.path.join(cn.EXPERIMENT_DIR, "workunits.txt")
with open(path, "r") as fd:
    workunit_strs = fd.readlines()
workunit_strs = [w if w[-1] == "l" else w[:-1] for w in workunit_strs]
workunits = [smt.Workunit.getFromStr(w) for w in workunit_strs]

In [5]:
# How many experiments were completed
len(DF)

16692

# Missing Experiments

In [6]:
set(["--".join(w.calcMultivaluedFactors()) for w in workunits])

{'biomodel_num--ts_instance'}

In [7]:
# Find what varies across workunits

In [8]:
# Combine workunits
agg_workunits = smt.ExtendedDict({k: [] for k in cn.SD_CONDITIONS})
for workunit in workunits[1:]:
    dct = workunit.removeExpansions()
    for key in dct.keys():
        # Handle bug where get [["differential_evolution"]]
        if isinstance(dct[key], str):
            values = set(agg_workunits[key]).union(dct[key])
        elif isinstance(dct[key], list):
            values = set(agg_workunits[key]).union(dct[key][0])
        agg_workunits[key] = list(values)
    print(agg_workunits)
if False:
    # Remove those known to vary
    for factor in [cn.SD_BIOMODEL_NUM, cn.SD_TS_INSTANCE]:
        del agg_workunits[factor]
    str(agg_workunits)

biomodel_num--a--l__method--differential_evolution__noise_mag--0.2__range_min_frac--0.5__range_max_frac--2__range_initial_frac--0.5__columns_deleted--0__max_fev--10000__ts_instance--a--l
biomodel_num--a--l__method--differential_evolution__noise_mag--0.2--0.1__range_min_frac--0.5__range_max_frac--2__range_initial_frac--0.5--0.8__columns_deleted--0__max_fev--10000__ts_instance--a--l
biomodel_num--a--l__method--differential_evolution__noise_mag--0.2--0.1__range_min_frac--0.5__range_max_frac--2__range_initial_frac--0.5--0.8__columns_deleted--0__max_fev--10000__ts_instance--a--l
biomodel_num--a--l__method--differential_evolution__noise_mag--0.2--0.1__range_min_frac--0.5__range_max_frac--2__range_initial_frac--0.5--0.8--1.2__columns_deleted--0__max_fev--10000__ts_instance--a--l
biomodel_num--a--l__method--differential_evolution__noise_mag--0.2--0.1__range_min_frac--0.5__range_max_frac--2__range_initial_frac--0.5--0.8--1.2__columns_deleted--0__max_fev--10000__ts_instance--a--l
biomodel_num--a

In [12]:
count_df = DF.groupby([cn.SD_NOISE_MAG, cn.SD_RANGE_INITIAL_FRAC, cn.SD_METHOD, cn.SD_MAX_FEV]).count()
df = count_df.compute()
ser = df[cn.SD_BIOMODEL_NUM]
ser

noise_mag  range_initial_frac  method                  max_fev
0.1        0.8                 differential_evolution  10000      1598
           1.2                 differential_evolution  10000      1500
           2.0                 differential_evolution  10000      1682
0.2        0.5                 differential_evolution  10000      1804
           1.2                 differential_evolution  10000      3238
0.1        1.5                 leastsq                 10000      3425
           2.0                 leastsq                 10000      3445
Name: biomodel_num, dtype: int64

In [10]:
dff = DF[[cn.SD_NOISE_MAG, cn.SD_RANGE_INITIAL_FRAC, cn.SD_METHOD]]
len(dff)

16692

In [11]:
len(dff.drop_duplicates())

7

In [21]:
?lhsmdu.sample

Signature:
lhsmdu.sample(
    numDimensions,
    numSamples,
    scalingFactor=5,
    numToAverage=2,
    randomSeed=42,
)
Docstring: Main LHS-MDU sampling function 
File:      ~/smarte/smt/lib/python3.6/site-packages/lhsmdu/__init__.py
Type:      function


In [19]:
import lhsmdu 
k = lhsmdu.sample(2, 20) # Latin Hypercube Sampling with multi-dimensional uniformity
k

matrix([[2.32101582e-01, 3.54206998e-01, 9.58081436e-01, 6.94927709e-01,
         2.80321453e-01, 4.59852581e-04, 4.55073577e-01, 5.33175088e-01,
         9.00253079e-01, 8.08040403e-01, 8.77436689e-01, 1.34594760e-01,
         4.32598063e-01, 5.61213465e-01, 7.35608961e-01, 7.61862454e-01,
         3.16269985e-01, 6.37324570e-01, 1.82481645e-01, 9.24611705e-02],
        [2.82880645e-01, 6.78415430e-01, 4.04683738e-01, 2.18385790e-01,
         1.63260118e-01, 4.62199482e-01, 7.98650528e-01, 8.19654886e-01,
         3.94602328e-01, 6.31556931e-01, 5.89740565e-01, 5.25131855e-01,
         3.28845194e-01, 8.74625885e-01, 9.59762149e-01, 3.61226058e-02,
         9.14038618e-01, 7.01215798e-01, 8.22736148e-02, 1.08855534e-01]])

In [13]:
from scipy.stats import qmc

sampler = qmc.LatinHypercube(d=2)

sample = sampler.random(n=5)

sample

ImportError: cannot import name 'qmc'